In [1]:
from goose import Goose
import unicodedata
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
import re


print("Imports Realizados!")

Imports Realizados!


In [2]:
def remover_naoLatin(string): 
    new_chars = []
    for char in string:
        if char == '\n':
            new_chars.append(' ')
            continue
        try:
            if unicodedata.name(unicode(char)).startswith(('LATIN', 'SPACE')):
                new_chars.append(char)
        except:
            continue
    return ''.join(new_chars)

In [27]:
def normalizar_palavra(palavra):

    # Unicode normalize transforma um caracter em seu equivalente em latin.
    palavraSemAcento = palavra.lower()
    try:
        nfkd = unicodedata.normalize('NFKD', palavra)
        palavraSemAcento = u"".join([c for c in nfkd if not unicodedata.combining(c)])
        palavraSemAcento = palavraSemAcento.lower()
    except:
        pass
    
    # Usa expressão regular para retornar a palavra apenas com números, letras e espaço
    return re.sub('[^a-zA-Z0-9 \\\]', '', palavraSemAcento)

In [4]:
def remover_stopwords(texto):
    c = []
    for i in stopwords.words("portuguese"):
        if len(i) > 1:
            c.append(i)
    saida = ""
    for palavra in c:
        saida = saida + " " + palavra

    x = normalizar_palavra(saida)  
    #print(x + "\n")

    p = ""
    for _p in texto.split():
        if _p not in x.split():
            if len(_p) > 1:
                p = p + _p + " "
    return p

In [5]:
def processar_texto(texto):
    texto = remover_naoLatin(texto)
    texto = normalizar_palavra(texto)
    return remover_stopwords(texto)
    

In [26]:
data = pd.read_csv('out.csv')
g = Goose()
teste = [["URL","PROCESSADO","CATEGORIA"]]

categorias = data['CATEGORIA']
urls = data['URL']

for url, categoria in zip(urls, categorias):
    print(url)
    print(categoria)
    artigo = g.extract(url=url)
    #texto = artigo.cleaned_text
    texto = artigo.meta_description
    lista_palavras = processar_texto(texto)
    teste.append([url, lista_palavras, categoria])
    


https://g1.globo.com/rj/rio-de-janeiro/noticia/rosinha-garotinho-deixa-prisao.ghtml
noticia
https://oglobo.globo.com/brasil/planilha-apreendida-com-aecio-revela-cota-de-cargos-do-tucano-no-governo-federal-22130663
noticia
https://g1.globo.com/al/alagoas/noticia/policia-federal-faz-operacao-contra-fraudes-nas-obras-do-canal-do-sertao.ghtml
noticia
https://g1.globo.com/rj/rio-de-janeiro/noticia/policia-do-rj-e-forcas-armadas-fazem-operacao-na-olha-do-governador.ghtml
noticia
https://g1.globo.com/economia/educacao-financeira/noticia/feirao-limpa-nome-da-serasa-termina-nesta-quinta-feira.ghtml
noticia
https://g1.globo.com/go/goias/noticia/mp-apura-suspeita-de-maus-tratos-contra-detentos-em-presidios-de-goias-videos-mostram-agressoes.ghtml
noticia
https://g1.globo.com/pr/parana/noticia/pf-faz-operacao-contra-quadrilha-especializada-em-explosoes-a-bancos-em-tres-estados.ghtml
noticia
https://g1.globo.com/sp/sao-paulo/noticia/ipva-fica-em-media-32-mais-barato-em-2018-em-sp.ghtml
noticia
https

In [28]:
df = pd.DataFrame(teste, columns=['URL','PROCESSADO','CATEGORIA'])
df = df.drop(df.index[[0]])

print(df)

X_train, X_test, y_train, y_test = train_test_split(df['PROCESSADO'], df['CATEGORIA'])

count_vector = CountVectorizer()
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)


                                                  URL  \
1   https://g1.globo.com/rj/rio-de-janeiro/noticia...   
2   https://oglobo.globo.com/brasil/planilha-apree...   
3   https://g1.globo.com/al/alagoas/noticia/polici...   
4   https://g1.globo.com/rj/rio-de-janeiro/noticia...   
5   https://g1.globo.com/economia/educacao-finance...   
6   https://g1.globo.com/go/goias/noticia/mp-apura...   
7   https://g1.globo.com/pr/parana/noticia/pf-faz-...   
8   https://g1.globo.com/sp/sao-paulo/noticia/ipva...   
9   https://g1.globo.com/mt/mato-grosso/noticia/co...   
10  https://g1.globo.com/pa/santarem-regiao/notici...   
11  https://g1.globo.com/pa/santarem-regiao/notici...   
12  https://g1.globo.com/mundo/noticia/talvez-um-d...   
13  https://g1.globo.com/sp/sao-paulo/noticia/poli...   
14  https://g1.globo.com/rj/rio-de-janeiro/noticia...   
15  https://g1.globo.com/ce/ceara/noticia/delegaci...   
16  https://g1.globo.com/monitor-da-violencia/noti...   
17  https://g1.globo.com/monito

In [38]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

print(X_test)
predictions = naive_bayes.predict(testing_data)


print(predictions)

1     rosinha saiu cadeia publica benfica minutos ca...
15    visita apuracao monitor violencia constatou si...
31                                                     
12    parentes menos esperancosos vez criticam marin...
26    apesar presidente conselho deliberativo mario ...
14    policia civil fez pericia cela cadeia jose fre...
22    emissario clube holandes deve apresentar propo...
28    conquista gaucha libertadores abre vaga compet...
29    nome treinador agrada elenco atleticomg apos r...
33                                                     
Name: PROCESSADO, dtype: object
['noticia' 'noticia' 'noticia' 'noticia' 'esporte' 'esporte' 'esporte'
 'esporte' 'esporte' 'noticia']
